In [8]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential, load_model

In [9]:
def generate(start):
    model = load_model('model.hdf5')
    notes=[]
    with open("notes", 'rb') as f:
        notes= pickle.load(f)

    pitchnames = sorted(set(notes))
    ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )
    n_vocab = len(set(notes))

    sequence_length = 20
    network_input = []

    for i in range(len(notes) - sequence_length):
        seq_in = notes[i : i+sequence_length] # contains 100 values
        network_input.append([ele_to_int[ch] for ch in seq_in])

    int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

    pattern = []
    prediction_output = []

    for i in start:
        pattern.append(i)
        prediction_output.append(int_to_ele[i])

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape 
        prediction_input = prediction_input/float(n_vocab)

        prediction =  model.predict(prediction_input, verbose=0)

        idx = np.argmax(prediction)
        result = int_to_ele[idx]
        prediction_output.append(result) 

        # Remove the first value, and append the recent value.. 
        # This way input is moving forward step-by-step with time..
        pattern.append(idx)
        pattern = pattern[1:]

    offset = 0 # Time
    output_notes = []

    for patterns in prediction_output:

        # if the pattern is a chord
        if ('+' in patterns) or patterns.isdigit():
            notes_in_chord = patterns.split('+')
            temp_notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))  # create Note object for each note in the chord
                new_note.storedInstrument = instrument.Piano()
                temp_notes.append(new_note)


            new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
            new_chord.offset = offset
            output_notes.append(new_chord)

        else:
                # if the pattern is a note
            new_note = note.Note(patterns)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_output_1.mid')
    return 


In [10]:
def show():
    notes=[]
    with open("notes", 'rb') as f:
        notes= pickle.load(f)

    pitchnames = sorted(set(notes))
    ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )
    for i in ele_to_int:
        print(ele_to_int[i],":",i)
    return

In [12]:
show()
start=[]
for i in range(20):
    x=int(input())
    if x>358:
        print("This note or chord doesnt exist")
        break
    start.append(i)
if len(start)==20:
    generate(start)

0 : 0
1 : 0+1
2 : 0+1+5
3 : 0+1+6
4 : 0+2
5 : 0+2+3+7
6 : 0+2+4+7
7 : 0+2+5
8 : 0+2+6
9 : 0+2+7
10 : 0+3
11 : 0+3+5
12 : 0+3+5+8
13 : 0+3+6
14 : 0+3+6+8
15 : 0+3+6+9
16 : 0+3+7
17 : 0+4
18 : 0+4+5
19 : 0+4+6
20 : 0+4+7
21 : 0+5
22 : 0+5+6
23 : 0+6
24 : 1
25 : 1+2
26 : 1+2+4+6+8+10
27 : 1+2+6
28 : 1+2+6+8
29 : 1+3
30 : 1+3+5
31 : 1+3+5+8
32 : 1+3+6
33 : 1+3+7
34 : 1+3+8
35 : 1+4
36 : 1+4+6
37 : 1+4+6+9
38 : 1+4+7
39 : 1+4+7+10
40 : 1+4+7+9
41 : 1+4+8
42 : 1+5
43 : 1+5+8
44 : 1+5+9
45 : 1+6
46 : 1+7
47 : 10
48 : 10+0
49 : 10+0+2+5
50 : 10+0+3
51 : 10+0+4
52 : 10+0+5
53 : 10+1
54 : 10+1+3
55 : 10+1+3+5+6
56 : 10+1+3+6
57 : 10+1+4
58 : 10+1+4+6
59 : 10+1+5
60 : 10+11
61 : 10+11+3
62 : 10+11+3+5
63 : 10+2
64 : 10+2+3
65 : 10+2+4
66 : 10+2+5
67 : 10+3
68 : 11
69 : 11+0
70 : 11+0+4
71 : 11+0+4+6
72 : 11+0+4+7
73 : 11+0+5
74 : 11+1
75 : 11+1+4
76 : 11+1+4+5
77 : 11+1+5
78 : 11+1+6
79 : 11+2
80 : 11+2+4
81 : 11+2+4+6
82 : 11+2+4+7
83 : 11+2+5
84 : 11+2+5+7
85 : 11+2+6
86 : 11+3
87 : 11+3+5
88 :